In [1]:
 

import numpy as np # linear algebra
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
LABEL_NAMES = ['Plane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

VALIDATION_SIZE = 10000

In [3]:
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()


In [4]:
LABEL_NAMES[y_train_all[1][0]]

'Truck'

In [5]:
x_train_all = x_train_all /255.0
x_test = x_test /255.0 

In [6]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_val.shape

(10000, 32, 32, 3)

In [7]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]
x_train.shape

(40000, 32, 32, 3)

In [8]:
 
y_train_ohe = np_utils.to_categorical(y_train, num_classes=10)
y_test_ohe = np_utils.to_categorical(y_test, num_classes=10)
y_val_ohe = np_utils.to_categorical(y_val, num_classes=10)

print(y_val_ohe)
y_val_ohe.shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


(10000, 10)

In [9]:
def create_cnn_model():
    image_input = Input(shape=(32, 32, 3))
    
    vgg_model  = VGG16(weights='imagenet',include_top=False, input_tensor=image_input)
    
    flatt = Flatten()(vgg_model.output)
    
    couche1 = Dense(128, activation='relu')(flatt) 
    couche1_normalization = BatchNormalization()(couche1)
    couche1_dropout = Dropout(0.2)(couche1_normalization)
    couche2 = Dense(64, activation='relu')(couche1_dropout)
    couche2_normalization = BatchNormalization()(couche2)
    output = Dense(10, activation='softmax', name='output')(couche2_normalization)     
    model = Model( image_input, output )
    return model

model = create_cnn_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [10]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
                metrics=['accuracy'])

In [11]:
 
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip= True)
es = EarlyStopping(patience=10, monitor='val_accuracy', mode='max')
mc = ModelCheckpoint('./weights.h5', monitor='val_accuracy', mode='max', save_best_only=True)

model.fit_generator(datagen.flow(x_train, y_train_ohe,batch_size = 32), steps_per_epoch = 800, epochs=20, validation_data=[x_val, y_val_ohe], callbacks = [es,mc])
# Load The Best weights in the ModelCheckpoint
model.load_weights('./weights.h5')

# Predict The Test
preds = model.predict(x_val)
score_test = accuracy_score( y_val, np.argmax(preds, axis=1) )

print (' LE SCORE DE TEST : ', score_test)
print('')


Epoch 1/20
800/800 [==============================] - 937s 1s/step - loss: 1.9931 - accuracy: 0.2016 - val_loss: 3.4398 - val_accuracy: 0.1185
Epoch 2/20
800/800 [==============================] - 928s 1s/step - loss: 1.8190 - accuracy: 0.2672 - val_loss: 9.2347 - val_accuracy: 0.2139
Epoch 3/20
800/800 [==============================] - 902s 1s/step - loss: 1.6055 - accuracy: 0.3587 - val_loss: 2.9505 - val_accuracy: 0.1663
Epoch 4/20
800/800 [==============================] - 918s 1s/step - loss: 1.4588 - accuracy: 0.4264 - val_loss: 1.4299 - val_accuracy: 0.4574
Epoch 5/20
800/800 [==============================] - 931s 1s/step - loss: 1.3709 - accuracy: 0.4671 - val_loss: 1.3112 - val_accuracy: 0.4940
Epoch 6/20
800/800 [==============================] - 918s 1s/step - loss: 1.2799 - accuracy: 0.5088 - val_loss: 1.3034 - val_accuracy: 0.4923
Epoch 7/20
800/800 [==============================] - 914s 1s/step - loss: 1.2388 - accuracy: 0.5316 - val_loss: 417.9929 - val_accuracy: 0.55

In [12]:
# after fit we can evaluate our model
_, evaluate = model.evaluate(x_test, y_test_ohe, verbose=1)
print('>%.3f' % (evaluate * 100.0))

10000/10000 [==============================] - 125s 12ms/step
>79.560
